In [1]:
# 首先导入需要的包
import os

import dgl
import dgl.function as fn
from dgl.nn.pytorch.softmax import edge_softmax

import torch as th
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import pandas as pd
import numpy as np

Using backend: pytorch


# DGL反欺诈项目代码实践

在这一任务里，我们会通过一个GNN模型的构建、探究和实验过程来学习如何使用DGL构建可被用于生产环境的代码。

在本教程里，我们会完成如下的任务：

1. 根据SOTA的算法论文，把计算公式转化成消息传递的模式；
2. 根据消息传递的模式，实现单层GNN的模块；
3. 叠加多层GNN模块，实现一个算法的模型；
4. 使用一个小样例数据来探究模型的运行机制；
5. 使用一个大图采样出的图数据，模拟模型的训练和推断。

注：大图采样出的数据的内容不具有实际意义，所以训练和推断的结果没有参考，仅用于演示。

## 算法论文的思路转换

阿里闲鱼团队在CIKM 2019上发了一篇进行[垃圾评论的检测算法](https://arxiv.org/abs/1908.10679)，被评为“最佳应用研究“论文。论文中利用了”用户“$\to$“评论”$\to$“商品”的二部图关系，通过含有Attention机制的特征传递方式，把“用户”和“商品”的特征加入“评论”的特征，帮助提升了对于“评论”的分类效果。
<img src='./assets/XY-Test-Data.png' width=25%>

### 算法的原理

- 通过Attention机制把“用户”+“评论”的特征发送给“商品”，并结合“商品”已有的特征，更新“商品”的特征；
- 通过Attention机制把“商品”+“评论”的特征发送给“用户”，并结合“用户”已有的特征，更新“用户”的特征；
- “用户”+“评论”+“商品”特征发给“评论”，并更新“评论”的特征；
- 最后使用“用户”$||$“笔记”$||$“评论”作为特征进行“评论”的分类。其中，$||$是$concatenate$的意思。

### 算法的核心公式
可以看到，闲鱼的算法是针对一个二部图的双向重复计算。所以下面的公式介绍只解释一个方向的公式。

- 对于一种点-“用户”的计算：

    1. 首先把一个用户评论过的所有**商品**的特征和评论的特征concatenate起来。
$$\mathcal{H}_{IE}^{l-1} = \{concat(h_i^{l-1}, h_e^{l-1}), \forall e=(u, i)\in E(u)\}$$


    2. 结合第1步里获取的合并特征，与“用户”的特征进行Attention的计算，获得“商品”+“评论”的带注意力组合的对“用户”的特征更新。
$$\mathcal{H}_{N(u)}^l = \sigma(W_U^l * ATTN_U(W_{AU}^{l-1} * h_u^{l-1}, W_{AIE}^{l-1} * \mathcal{H}_{IE}^{l-1}))$$
其中，
$$ATTN_U(W_{AU}^{l-1} * h_u^{l-1}, W_{AIE}^{l-1} * \mathcal{H}_{IE}^{l-1}) \implies \mathcal{H}_{IE}^l$$


    3. 使用“用户”自身的特征做非线性变换，然后与注意力更新后的特征进行组合，获得下一层的隐藏特征
$$h_u^l = concat(V_U^l * h_u^{l-1}, \mathcal{H}_{N(u)}^l)$$


在这计算过程中，需要实现对于Attention的计算，原文中并没有给出。下面是按照[Attention is all you need](https://arxiv.org/abs/1706.03762)里面的原理给出计算公式。
    
    首先给出计算Attention前的符号定义：
$$\hat{h}_{u-1}^{l-1} = W_{AU}^{l-1} * h_u^{l-1},    \hat{\mathcal{H}}_{IE}^{l-1}=W_{AIE}^{l-1} * \mathcal{H}_{IE}^{l-1}$$

    a.1 通过点积计算出“商品”和“评价”对于“用户”的注意力原始值
$$A_{attn\_u} = \hat{h}_{u-1}^{l-1} \odot \hat{\mathcal{H}}_{IE}^{l-1}$$
    
    a.2 按照每个用户的“评论”边进行$softmax$计算。
$$A_{attn\_u} =softmax(A_{attn\_u} )$$

    a.3 把上一步softmax计算出的注意力系数和第1步合并的特征值广播相乘
$$\mathcal{\hat{H}}_{IE}^l = \mathcal{H}_{IE}^{l-1} * A_{attn\_u}$$

    a.4 最后一步把a.3的结果，按列求和，聚合成带注意力组合的对“用户”的特征更新
$$\mathcal{H}_{IE}^l = sum(\mathcal{\hat{H}}_{IE}^l,  dim=-1)$$
    
    
    
- 对边-“评论“的计算：
$$h_e^l = \sigma(W_E^l * concat(h_e^{l-1}, h_u^{l-1}, h_i^{l-1}))$$


### 按消息传递的模式构建模型



下面首先构建GNN的一层。

In [2]:
class layer(nn.Module):
    """
    This layer is designed specifically for user Xianyu Graph algorithm.
    """
    def __init__(self, d_u_in, d_u_out, d_e_in, d_e_out, d_i_in, d_i_out):
        super(layer, self).__init__()
        self.act = F.relu

        # 上面公式里对“评论”的三个权重，这里使用了先线性计算再concat的同质方式 
        self.W_Ee = nn.Linear(d_e_in, d_e_out, bias=False)
        self.W_Eu = nn.Linear(d_u_in, d_e_out, bias=False)
        self.W_Ei = nn.Linear(d_i_in, d_e_out, bias=False)

        # 上面公式里第2步的第二个和第三个权重，这里是双向的：u->i 和 i->u
        # 1. i -> u 的权重
        self.d_attn_ie_in = d_e_in + d_i_in
        self.d_attn_u_out = self.d_attn_ie_in
        self.W_ATTN_ie = nn.Linear(self.d_attn_ie_in, self.d_attn_u_out, bias=False)
        self.W_ATTN_u = nn.Linear(d_u_in, self.d_attn_u_out, bias=False)

        # 2. u -> i 的权重
        self.d_attn_ue_in = d_e_in + d_u_in
        self.d_attn_i_out = self.d_attn_u_out
        self.W_ATTN_ue = nn.Linear(self.d_attn_ue_in, self.d_attn_i_out, bias=False)
        self.W_ATTN_i = nn.Linear(d_i_in, self.d_attn_i_out, bias=False)

        # 上面公式第2步里的第一权重，也是分u和i的
        self.d_wu_in = self.d_attn_u_out
        self.d_wu_out = int(d_u_out / 2)
        self.W_nu = nn.Linear(self.d_wu_in, self.d_wu_out, bias=False)

        self.d_wi_in = self.d_attn_i_out
        self.d_wi_out = int(d_i_out / 2)
        self.W_ni = nn.Linear(self.d_wi_in, self.d_wi_out, bias=False)

        # 上面第3步里的权重
        self.d_vu_out = d_u_out - self.d_wu_out
        self.W_u = nn.Linear(d_u_in, self.d_vu_out, bias=False)

        self.d_vi_out = d_i_out - self.d_wi_out
        self.W_i = nn.Linear(d_i_in, self.d_vi_out, bias=False)
        
        
    def forward(self, graph, u_feats, e_feats, i_feats):
        """
        Specificlly For this algorithm, feat_dict has 3 types of features:
        'User': l-1 layer's user features, in dict {'u': features}
        'Edge': l-1 layer's edge features, in dict {'e': features}
        'Item': l-1 layer's note features, in dict {'i': features}

        This version, we have one edge but two types:
            - 'comment_on'
            - 'commented_by'

        :param graph: bi-partitie
        
        :return:
        """
        # L-1层的特征赋值
        graph.nodes['user'].data['u'] = u_feats
        graph.nodes['item'].data['i'] = i_feats
        graph.edges['comment_on'].data['e'] = e_feats
        graph.edges['commented_by'].data['e'] = e_feats

        # 上面公式里的第1步concat计算
        graph.apply_edges(lambda edges: {'h_ie': th.cat([edges.src['i'], edges.data['e']], dim=-1)}, etype='commented_by')
        graph.apply_edges(lambda edges: {'h_ue': th.cat([edges.src['u'], edges.data['e']], dim=-1)}, etype='comment_on')

        # 注意力的计算部分
        graph.nodes['user'].data['h_attnu'] = self.W_ATTN_u(u_feats)
        graph.nodes['item'].data['h_attni'] = self.W_ATTN_i(i_feats)
        graph.edges['commented_by'].data['h_attne'] = self.W_ATTN_ie(graph.edges['commented_by'].data['h_ie'])
        graph.edges['comment_on'].data['h_attne'] = self.W_ATTN_ue(graph.edges['comment_on'].data['h_ue'])

        # a.1: 点积计算
        graph.apply_edges(fn.e_dot_v('h_attne', 'h_attnu', 'edotv'), etype='commented_by')
        graph.apply_edges(fn.e_dot_v('h_attne', 'h_attni', 'edotv'), etype='comment_on')

        # a.2. 按边的softmax计算
        graph.edges['commented_by'].data['sfm'] = edge_softmax(graph['commented_by'], graph.edges['commented_by'].data['edotv'])
        graph.edges['comment_on'].data['sfm'] = edge_softmax(graph['comment_on'], graph.edges['comment_on'].data['edotv'])

        # a.3. 广播softmax值到每条边
        graph.apply_edges(lambda edges: {'attn': edges.data['h_attne'] * edges.data['sfm'].unsqueeze(dim=0).T},
                          etype='commented_by')
        graph.apply_edges(lambda edges: {'attn': edges.data['h_attne'] * edges.data['sfm'].unsqueeze(dim=0).T},
                          etype='comment_on')

        # a.4. 按列求和，聚合注意力加权后的值到端点
        graph.update_all(fn.copy_e('attn', 'm'), fn.sum('m', 'agg_u'), etype='commented_by')
        graph.update_all(fn.copy_e('attn', 'm'), fn.sum('m', 'agg_i'), etype='comment_on')

        # 完成上面公式第2步
        graph.nodes['user'].data['h_nu'] = self.act(self.W_nu(graph.nodes['user'].data['agg_u']))
        graph.nodes['item'].data['h_ni'] = self.act(self.W_ni(graph.nodes['item'].data['agg_i']))

        # 完成上面公式3的计算
        graph.nodes['user'].data['u'] = th.cat([self.W_u(u_feats), graph.nodes['user'].data['h_nu']], dim=-1)
        graph.nodes['item'].data['i'] = th.cat([self.W_i(i_feats), graph.nodes['item'].data['h_ni']], dim=-1)

        # 上面公式里对“评论”的计算，这里是先矩阵相乘，再进行concat。
        # 首先，完成矩阵相乘
        graph.edges['comment_on'].data['h_e'] = self.W_Ee(e_feats)
        graph.edges['commented_by'].data['h_e'] = self.W_Ee(e_feats)
        graph.nodes['user'].data['h_u4e'] = self.W_Eu(u_feats)
        graph.nodes['item'].data['h_i4e'] = self.W_Ei(i_feats)

        # 然后，利用边的消息传递完成concat操作
        graph.apply_edges(fn.u_add_e('h_u4e', 'h_e', 'h_ue'), etype='comment_on')
        graph.apply_edges(fn.e_add_v('h_ue', 'h_i4e', 'e'), etype='comment_on')
        graph.edges['comment_on'].data['e'] = self.act(graph.edges['comment_on'].data['e'])

        graph.edges['commented_by'].data['e'] = graph.edges['comment_on'].data['e']

        # 输出L层的特征
        u_feats = graph.nodes['user'].data['u']
        e_feats = graph.edges['comment_on'].data['e']
        i_feats = graph.nodes['item'].data['i']

        return u_feats, e_feats, i_feats

### 堆叠多层形成一个GNN的模型

这里我们按照通常GNN的设计，构建2层的算法模型。并再最后接一个全联接层，做一次2分类的logit输出。

In [3]:
class Algorithm_Model(nn.Module):

    def __init__(self, u_in_dim, e_in_dim, i_in_dim,
                 hidden_dim, out_dim_ratio=2, num_class=2):

        super(Algorithm_Model, self).__init__()
        
        # 定义模型内部使用的隐藏层维度，这里按照2倍来进行扩展隐藏层的维度
        u_out_dim = u_in_dim * out_dim_ratio
        e_out_dim = e_in_dim * out_dim_ratio
        i_out_dim = i_in_dim * out_dim_ratio
        g_out_dim = u_out_dim + e_out_dim + i_out_dim
        
        # 定义2层的GNN
        self.layer_1 = layer(u_in_dim, hidden_dim, e_in_dim, hidden_dim, i_in_dim, hidden_dim)
        self.layer_2 = layer(hidden_dim, u_out_dim, hidden_dim, e_out_dim, hidden_dim, i_out_dim)
        # 定义最后的输出层
        self.output = nn.Linear(g_out_dim, out_features=num_class)
        
    def forward(self, graph, u_features, e_features, i_features):
        
        h_u, h_e, h_i = self.layer_1(graph, u_features, e_features, i_features)
        h_u = F.relu(h_u)
        h_e = F.relu(h_e)
        h_i = F.relu(h_i)
        h_u, h_e, h_i = self.layer_2(graph, h_u, h_e, h_i)

        # 使用DGL的消息传递机制来concat“用户”，“商品”和“评论”的特征
        # 先赋值
        graph.nodes['user'].data['u'] = h_u
        graph.nodes['item'].data['i'] = h_i
        graph.edges['comment_on'].data['e'] = h_e

        # 按照边来concat两个端点的特征
        graph.apply_edges(lambda edges:
                          {'output': th.cat([edges.src['u'], edges.data['e'], edges.dst['i']], dim=-1)}, 
                          etype='comment_on')

        # 最后的分类输出层计算
        output = graph.edges['comment_on'].data['output']
        logits = self.output(output)
        
        return logits

## 使用小样例数据来对模型的内部运作进行探究

这里使用和原论文的例图结构一样一个极小数据，帮助了解模型的一层的内部运作机制，方便进一步理解算法和DGL的运作机制。
<img src='./assets/XY-Test-Data.png' width=50%>

In [4]:
def build_xy_graph(u, v):

    graph = dgl.heterograph(
        {('user', 'comment_on', 'item'): (u, v),
         ('item', 'commented_by', 'user'): (v, u)}
    )

    return graph

u = th.tensor([0,1,2,1,1,3,4,4])
v = th.tensor([0,0,0,1,2,1,1,2])

xy_graph = build_xy_graph(u, v)

#### 下面给这个图的节点和边赋一些随机的值

- 用户节点: 6d，用户特征的值和用户自己对应的ID一致，例如，节点0是\[0,0,0,0,0,0\]，节点1是\[1,1,1,1,1,1\]，以此类推。
- 商品节点: 3d, 商品特征的值和商品自己对应的ID一致，但是负数，例如，节点0是\[0,0,0\]，节点1是\[-1,-1,-1\]，以此类推。
- 评论边:   7d, 对于“comment_on”和“commented_by”类型的边赋相同的值，特征的值和边对应的ID一致，但缩小1/10，例如，边0是\[0,0,0,0,0,0,0\]，边1是\[0.1,0.1,0.1,0.1,0.1,0.1,0.1\]，以此类推。

In [5]:
user_feats = th.from_numpy(np.ones([5,6]) * np.arange(5).reshape(5,1)).float()
xy_graph.nodes['user'].data['u'] = user_feats
print('用户输入特征：\n', xy_graph.nodes['user'].data)

item_feats = th.from_numpy(np.ones([3,3]) * np.arange(3).reshape(3,1) * -1).float()
xy_graph.nodes['item'].data['i'] = item_feats
print('商品输入特征：\n', xy_graph.nodes['item'].data)

edge_feats = th.from_numpy(np.ones([8,7]) * np.arange(8).reshape(8,1) * 0.1).float()
xy_graph.edges['comment_on'].data['e'] = edge_feats
xy_graph.edges['commented_by'].data['e'] = edge_feats
print('评论的边的特征：\n', xy_graph.edges['comment_on'].data)

用户输入特征：
 {'u': tensor([[0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2., 2.],
        [3., 3., 3., 3., 3., 3.],
        [4., 4., 4., 4., 4., 4.]])}
商品输入特征：
 {'i': tensor([[-0., -0., -0.],
        [-1., -1., -1.],
        [-2., -2., -2.]])}
评论的边的特征：
 {'e': tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000],
        [0.4000, 0.4000, 0.4000, 0.4000, 0.4000, 0.4000, 0.4000],
        [0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
        [0.6000, 0.6000, 0.6000, 0.6000, 0.6000, 0.6000, 0.6000],
        [0.7000, 0.7000, 0.7000, 0.7000, 0.7000, 0.7000, 0.7000]])}


#### 初始化一层的实例

这里会初始化我们定义一层layer，按照上面layer类init方法的定义，用户特征输入维度是5，商品特征输入维度是3，评论边特征输入维度是7。相应的隐藏维度定为20。

In [6]:
test_layer = layer(d_u_in=6, d_u_out=20, d_e_in=7, d_e_out=20, d_i_in=3, d_i_out=20)

# 前向传播
user_out, edge_out, item_out = test_layer(xy_graph, user_feats, edge_feats, item_feats)

print("新一层用户的特征维度\n", user_out.shape)
print("新一层用户的特征\n", user_out)

print("新一层评论边的特征维度\n", edge_out.shape)
print("新一层评论边的特征\n", edge_out)

print("新一层商品的特征维度\n", item_out.shape)
print("新一层商品的特征\n", item_out)

# 探究中间计算结果来进一步探究内部
print('完成一层前向传播计算后的评论的边的特征：\n', xy_graph.edges['comment_on'].data)

# TODO: 您可以使用print方法来查看xy_graph的内部的数据信息来进一步了解每一步计算的内部机制。

新一层用户的特征维度
 torch.Size([5, 20])
新一层用户的特征
 tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.5511, -0.2027,  0.7250,  0.0562, -0.2796,  1.1621, -0.2732,  0.9873,
          0.0972, -0.2080,  0.0805,  0.1811,  0.0000,  0.0000,  0.0000,  0.0000,
          0.1064,  0.0143,  0.0000,  0.0000],
        [ 1.1021, -0.4055,  1.4499,  0.1125, -0.5593,  2.3241, -0.5464,  1.9746,
          0.1944, -0.4159,  0.0000,  0.0000,  0.0000,  0.0000,  0.0381,  0.0000,
          0.1561,  0.0621,  0.0000,  0.0632],
        [ 1.6532, -0.6082,  2.1749,  0.1687, -0.8389,  3.4862, -0.8196,  2.9618,
          0.2917, -0.6239,  0.0701,  0.2930,  0.0000,  0.0000,  0.0000,  0.0000,
          0.3022,  0.0729,  0.0000,  0.0000],
        [ 2.2043, -0.8109,  2.8999,  0.2250, -1.1185,  4.6482, -1.0928,  3.9491,
          0.3889, -0.8318,  0.0466,  0.3101, 

## 实验数据全图训练和推断

为了展示如何完成对于这个模型的训练和推断，这里会使用从一个稍大的图数据。

请注意：其中的标签是随机生成，仅用于演示。

In [7]:
# 从本地文件读取实验用图数据文件
data_path = "./example_graph"
src = pd.read_csv(os.path.join(data_path, 'e_src.csv'))
dst = pd.read_csv(os.path.join(data_path, 'e_dst.csv'))
u_feats = pd.read_csv(os.path.join(data_path, 'user.csv'))
e_feats = pd.read_csv(os.path.join(data_path, 'edge.csv'))
i_feats = pd.read_csv(os.path.join(data_path, 'item.csv'))

print("样例图有:{}条边".format(e_feats.shape[0]))

print("用户特征维度:{}".format(u_feats.shape))
print("商品特征维度:{}".format(i_feats.shape))
print("评论特征维度:{}".format(e_feats.shape))

样例图有:15345条边
用户特征维度:(11832, 6)
商品特征维度:(3136, 3)
评论特征维度:(15345, 7)


In [8]:
# 使用导入源和目标节点数据构建模型所需的DGL的图
src_tensor = th.from_numpy(src.e_src.to_numpy())
dst_tensor = th.from_numpy(dst.e_dst.to_numpy())
graph = build_xy_graph(src_tensor, dst_tensor)

# 生成可供模型使用的点和边的特征数据
user_feats = th.from_numpy(u_feats.to_numpy()).float()
edge_feats = th.from_numpy(e_feats.to_numpy()).float()
item_feats = th.from_numpy(i_feats.to_numpy()).float()

# 出于演示目的，这里随机选择1000条边，并给其中900条边赋0，表示为正常的（白）边；100条赋1，表示有疑问的（黑）边。
num_of_edges = e_feats.shape[0]

target_idx = np.random.choice(num_of_edges, 1000, replace=False)
white_idx = target_idx[:900]
black_idx = target_idx[900:]

target_label = th.cat([th.zeros(900), th.ones(100)]).long()

### 训练过程

对于我们的实验数据，由于并不是很大，可以放进内存和显存，所以我们会采用全图训练。采样的方式可以参考大图采样的notebook。更多的DGL大图采样的例子可以在我们的[github](https://github.com/dmlc/dgl/tree/master/examples/pytorch/graphsage)里面找到。

In [9]:
# --------------- 1. 构建GNN模型 ------------------ #
# 根据我们的数据的维度构建模型
model = Algorithm_Model(u_in_dim=6, e_in_dim=7, i_in_dim=3, hidden_dim=20)

# --------------- 2. 构建损失函数和优化器 ----------- #
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam([{'params': model.parameters(), 'lr':0.001, 'weight_decay':5e-4}])

# --------------- 3. 进行模型训练 ------------------ #
MAX_EPOCH = 30
for epoch in range(MAX_EPOCH):
    # 设置成训练模式
    model.train()
    
    # 前向传播
    logits = model(graph, user_feats, edge_feats, item_feats)
    
    # 提取标签的边
    train_logits = logits[target_idx]
    
    # 计算loss
    train_loss = loss_fn(train_logits, target_label)

    # 计算指标
    train_pred = train_logits.argmax(1)
    train_acc = (train_pred == target_label).float().mean()

    # 反向传播
    opt.zero_grad()
    train_loss.backward()
    opt.step()

    model.eval()
    # 这里省略了validation的代码，用户应该能简单的加入相应的代码

    print('In {:03d} Epoch: train_loss: {:.4f} acc: {:.4f}'.
        format(epoch, train_loss, train_acc))

# --------------- 4. 保存模型 ------------------ #
model_para_dict = model.state_dict()
model_path = os.path.join('./', 'model' + '.pth')
th.save(model_para_dict, model_path)
print("\nModel parameters are saved to {}".format(model_path))

In 000 Epoch: train_loss: 0.7245 acc: 0.1750
In 001 Epoch: train_loss: 0.7113 acc: 0.2500
In 002 Epoch: train_loss: 0.6981 acc: 0.3750
In 003 Epoch: train_loss: 0.6850 acc: 0.5760
In 004 Epoch: train_loss: 0.6720 acc: 0.7930
In 005 Epoch: train_loss: 0.6592 acc: 0.9000
In 006 Epoch: train_loss: 0.6465 acc: 0.9000
In 007 Epoch: train_loss: 0.6339 acc: 0.9000
In 008 Epoch: train_loss: 0.6212 acc: 0.9000
In 009 Epoch: train_loss: 0.6087 acc: 0.9000
In 010 Epoch: train_loss: 0.5961 acc: 0.9000
In 011 Epoch: train_loss: 0.5837 acc: 0.9000
In 012 Epoch: train_loss: 0.5714 acc: 0.9000
In 013 Epoch: train_loss: 0.5592 acc: 0.9000
In 014 Epoch: train_loss: 0.5470 acc: 0.9000
In 015 Epoch: train_loss: 0.5349 acc: 0.9000
In 016 Epoch: train_loss: 0.5229 acc: 0.9000
In 017 Epoch: train_loss: 0.5110 acc: 0.9000
In 018 Epoch: train_loss: 0.4992 acc: 0.9000
In 019 Epoch: train_loss: 0.4875 acc: 0.9000
In 020 Epoch: train_loss: 0.4757 acc: 0.9000
In 021 Epoch: train_loss: 0.4639 acc: 0.9000
In 022 Epo

### 推断过程

由于我们的模型是一个inductive的模型，即不依赖于原图的结构，所以可以对新加入的边进行预测推断，包括原图已经发生改变的情况。

出于演示目的，我们人为地在原图里增加1条新的边和新边的特征。为简单起见，这里是对已经存在的2个点增加新边，而不再创建新的点。

In [10]:
# 在源和目标点列表的最后加入新的边
new_src_tensor = th.cat([src_tensor, th.tensor([11509])])
new_dst_tensor = th.cat([dst_tensor, th.tensor([142])])

# 在边特征的最后加入新的边的特性
new_edge_feats = th.cat([edge_feats, th.tensor([[0.257663393,1,0,0,1,0,0]])])

torch.Size([15346, 7])


In [11]:
# 重构我们的图
new_graph = build_xy_graph(new_src_tensor, new_dst_tensor)
print(new_graph.number_of_nodes('user'))
print(new_graph.number_of_nodes('item'))
print(new_graph.number_of_edges('comment_on'))

11832
3136
15346


In [14]:
# 导入保存的模型
stat_dict = th.load(model_path, map_location=th.device('cpu'))

xy_model = Algorithm_Model(u_in_dim=6, e_in_dim=7, i_in_dim=3, hidden_dim=20)
xy_model.load_state_dict(stat_dict)

# 进行预测推断
logits = xy_model(new_graph, user_feats, new_edge_feats, item_feats)
target_logits = logits[-1:]

pred = logits.argmax(1)
target_pred = pred[-1:]
print("我们新边的预测logits是: {}".format(target_logits.detach()))
print("我们新边的预测标签是: {}".format(target_pred))

我们新边的预测logits是: tensor([[ 0.3467, -0.9935]])
我们新边的预测标签是: tensor([0])
